In [5]:
from scipy import stats
%matplotlib widget
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import matplotlib.cm as cm
from Funciones_auxiliares import Mat_to_dataframe,see_teams,get_cluster_indexs,plot_branch,plot_teams,find_mix
from scipy import cluster

import scipy as sp
import math


### Importamos la data

In [6]:
plt.close('all')
plt.clf()
plt.close()
data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
del data['Bulk']
Initial = len(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Separamos en Test y Train por sessiones

In [7]:
T = True
while T:
    Sessions_list = data.PatientExperiment.unique()
    select_random_sessions = np.random.choice(Sessions_list,(10,6), replace = False)
    Dfs = {}
    for i,sessions in enumerate(select_random_sessions):
        Dfs[i] = data[data.PatientExperiment.isin(sessions)]
    lenghts = np.array([len(i) for i in Dfs.values()])
    if np.all(lenghts > 80): T = False
    
for key,values in Dfs.items():
    print(len(values))

205
222
133
157
147
112
237
86
140
138


Calculamos la matriz de correlacion

## No tengo que entrenar nada, asumimos que el template matching para train es perfecto

Distancia Correlation

In [12]:
def Train_test_noise(data):
    Sessions_list = data.PatientExperiment.unique()
    select_random_sessions = np.random.randint(0, len(Sessions_list), size=int(len(Sessions_list)/2))
    Train_list = Sessions_list[select_random_sessions]
    Train = data[data.PatientExperiment.isin(Train_list)]
    Test = data[~data.PatientExperiment.isin(Train_list)]
    Train.reset_index(inplace = True,drop = True)
    Test.reset_index(inplace = True,drop = True)
    
    df_train = Train.copy()
    df_test = Test.copy()
    
    Noise = df_train[df_train.bNoise == 1]
    Neuron = df_train[df_train.bNoise == 0]
    MU = df_train[df_train.bNoise == 2]

    df_train = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
    t = df_train[df_train.bNoise == 1].index[0]

    Noise = df_test[df_test.bNoise == 1]
    Neuron = df_test[df_test.bNoise == 0]
    MU = df_test[df_test.bNoise == 2]

    df_test = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
    t2 = df_test[df_test.bNoise == 1].index[0]

    
    return df_train,df_test,t,t2

df_train,df_test,t,t2 = Train_test_noise(data)
print(len(df_train),len(df_test))

310 1260




## Testeamos

In [13]:
from scipy.spatial.distance import squareform, pdist

def compute_dist(df_test,df_train):
    df = pd.concat([df_test,df_train],ignore_index=True,sort = True)
    r = len(df_train)
    s = len(df_test)
    Dist = pd.DataFrame(squareform(pdist(np.array(df.loc[:, 'Mean'].to_list()),metric = 'correlation')),columns=df.index,index=df.index).values
    Dist = Dist[s:,:r]
    return Dist

Dist = compute_dist(df_test,df_train)
print(len(df_train))
print(len(df_test))

310
1260


Vemos la matriz de distancia, la linea negra separa neuron-noise

In [14]:
plt.figure()
plt.imshow(Dist)
plt.colorbar()
plt.hlines(df_test[df_test.bNoise == 1].index[0],0,len(Dist)-1)
plt.vlines(df_train[df_train.bNoise == 1].index[0],0,len(Dist)-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Ahora tenemos una matriz de distancias. Cada fila representa la distancia de un template test a cada uno de los templates de train. Computamos el minimo ( o n minimos en cuyo caso hacemos una votacion), ademas, si es ruido pedimos que la distancia sea menor a un cierto threshold, y nos fijamos si es neuron o noise, entonces asignamos ese valor.

In [15]:
def create_label(Dist,t,threshold = .5):
    number_votes = 1
    Minn = [np.argsort(i)[:number_votes] for i in Dist[:]]
    label = np.zeros(len(Minn),dtype = int)
    votes = len(Minn[0])
    for i,element in enumerate(Minn):
        votes_noise = len(np.where((element > t) & (Dist[i,element] < threshold))[0])
        if votes_noise >= int(votes/2) + 1: label[i] = 1
    return label

label = create_label(Dist,t)

In [16]:
neurons = label[:t2]
noises = label[t2:]

total_neurons = len(neurons)
total_noises = len(noises)
neurons_as_noise = sum(neurons)
noises_found = sum(noises)

print('neurons as noise = ',neurons_as_noise/total_neurons*100,'({})'.format(neurons_as_noise))
print('Noise found = ', noises_found/total_noises*100, '({})'.format(noises_found))


neurons as noise =  69.03225806451613 (214)


ZeroDivisionError: division by zero

In [21]:
def do_one_time(number_votes = 3):
    df_train,df_test,t2,t = Train_test_noise(data)
    Dist = compute_dist(df_test,df_train)
    label = create_label(Dist,t,threshold=.5)

    neurons = label[:t2]
    noises = label[t2:]

    total_neurons = len(neurons)
    print('total neurons in test = ',total_neurons,'. This represents ', total_neurons/len(label)*100,' of the test size')
    total_noises = len(noises)
    print('total neurons in test = ',total_noises,'. This represents ', total_noises/len(label)*100,' of the test size')

    
    total_found = sum(label)
    print( 'The algorithm labeled ',total_found, 'clusters as noise')
    
    neurons_as_noise = sum(neurons)
    print('The algorithm incorrectely labeled',round(neurons_as_noise/total_found*100,2), 'of the test (falses_positives)')
    noises_found = sum(noises)
    print('The algorithm correctely labeled',round(noises_found/total_found*100,2), 'of the test (True Noises)')
    
    print('\n')
    print('\n')

    
    falses_positives = neurons_as_noise/total_found
    noise_found = noises_found/total_noises
    
    return falses_positives,noise_found

In [22]:
falses_positives_list = []
noise_found_list= []
for i in range(20):
    a,b =  do_one_time(number_votes=1)
    falses_positives_list.append(a)
    noise_found_list.append(b)

total neurons in test =  247 . This represents  35.38681948424069  of the test size
total neurons in test =  451 . This represents  64.61318051575931  of the test size
The algorithm labeled  432 clusters as noise
The algorithm incorrectely labeled 11.34 of the test (falses_positives)
The algorithm correctely labeled 88.66 of the test (True Noises)




total neurons in test =  235 . This represents  36.321483771251934  of the test size
total neurons in test =  412 . This represents  63.67851622874807  of the test size
The algorithm labeled  385 clusters as noise
The algorithm incorrectely labeled 11.43 of the test (falses_positives)
The algorithm correctely labeled 88.57 of the test (True Noises)




total neurons in test =  276 . This represents  37.653478854024556  of the test size
total neurons in test =  457 . This represents  62.346521145975444  of the test size
The algorithm labeled  424 clusters as noise
The algorithm incorrectely labeled 11.08 of the test (falses_positives)
The 

In [23]:
print(np.mean(falses_positives_list),'pm',np.std(falses_positives_list))
print(np.mean(noise_found_list),'pm',np.std(noise_found_list))

0.09044982201240623 pm 0.07169026155526116
0.6273772411668748 pm 0.2691610005800522
